In [ ]:
cache="mg"
def load_prec(filename):
    f= open("cache/"+filename+"_acc.txt","r")
    acc=[(i[:(-1 if len(i)<4 else -2)]).split(",") for i in f]
    f.close()
    f= open("cache/"+filename+"_acc.txt","r")
    t=[(i[:(-1 if len(i)<4 else -2)]).split(",") for i in f]
    f.close()
    return acc,t


In [ ]:

from sklearn.datasets import fetch_openml
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd

def load_dataset(partition):
    X,y = fetch_openml('mnist_784', version=1, return_X_y=True)
    y = y.astype(int)[:int(len(X)*partition//1)]
    X = X/255
    X=X[:int(len(X)*partition//1)]
    print(len(X))
    return X,y

In [ ]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler

X,y= load_dataset(.33)

In [ ]:
from matplotlib import pyplot as plt
print(len(X.columns))
n_components=300
pca=PCA(n_components=n_components)
pca.fit(X)

ks=np.arange(1,n_components+1)
plt.plot(ks,np.cumsum(pca.explained_variance_ratio_))
Xs=pca.transform(X)

In [ ]:
from sklearn.cluster import SpectralClustering
from sklearn.metrics import rand_score
from tqdm import tqdm
import csv
import time
from google.colab import drive
drive.mount('/content/drive')
path='/content/drive/My Drive/ms.txt'
data=[]
try:
  with open(path,"r") as f:
      n_cluster=eval(f.readln())
      pca_dimension=eval(f.readln())
      while(True):
        l=f.readln()
        if l=="": break
        data.append()

except:
  print("no_cache")
  n_cluster=list(np.arange(5,16))
  pca_dimension=[10,211,20]
  pca_dimension.insert(0,2)
  w=open(path,"w")
  w.writeln(str(n_cluster))
  w.writeln(str(pca_dimension))


for i,pc in enumerate(pca_dimension):
    pca=PCA(n_components=pc)
    pca.fit(X)
    Xs=pca.transform(X)
    for j,n in tqdm(enumerate(n_cluster)):

        sti=time.time()
        sc=SpectralClustering(n,eigen_tol=0.00002,n_jobs=-1)
        pred=sc.fit_predict(Xs)
        ts=time.time()-sti
        t[i].append(ts)
        t_out.write(str(ts)+',')
        rs=rand_score(y,pred)
        print(rs)
        acc_out.write(str(rs)+',')
        acc[i].append(rs)
        if i>=len(acc)-1 and j==len(n_cluster)-1:
            print("end_row")
            acc_out.write("\n")
            t_out.write("\n")
        acc_out.close()
        t_out.close()

    print(acc)